In [1]:
# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax, Adagrad
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/feature_selection_positive.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [3]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
def create_model(optimizer='rmsprop'):
    tf.set_random_seed(42)
    c=0.4
    # make keras model
    start = time.time()
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(1024, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = KerasClassifier(
    build_fn=create_model, 
    verbose=2, 
    epochs=100, 
    batch_size=1000
)

In [6]:
p = [
        'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 
        'Adam', 'Adamax', 'Nadam'
]  # optimizers

params = dict(
    optimizer=p
             )

In [7]:
grid = GridSearchCV(
    estimator=model, 
    param_grid=params
)

In [8]:
grid_result = grid.fit(X_train, y_train_for_keras)

Epoch 1/100
 - 2s - loss: 1.9306 - acc: 0.2632
Epoch 2/100
 - 0s - loss: 1.9362 - acc: 0.2464
Epoch 3/100
 - 0s - loss: 1.8529 - acc: 0.2699
Epoch 4/100
 - 0s - loss: 1.9301 - acc: 0.2699
Epoch 5/100
 - 0s - loss: 1.7973 - acc: 0.2777
Epoch 6/100
 - 0s - loss: 1.8638 - acc: 0.2643
Epoch 7/100
 - 0s - loss: 1.8397 - acc: 0.2486
Epoch 8/100
 - 0s - loss: 1.8643 - acc: 0.2452
Epoch 9/100
 - 0s - loss: 1.8046 - acc: 0.2688
Epoch 10/100
 - 0s - loss: 1.7342 - acc: 0.2777
Epoch 11/100
 - 0s - loss: 1.7178 - acc: 0.2688
Epoch 12/100
 - 0s - loss: 1.6839 - acc: 0.2699
Epoch 13/100
 - 0s - loss: 1.7187 - acc: 0.2721
Epoch 14/100
 - 0s - loss: 1.7408 - acc: 0.2800
Epoch 15/100
 - 0s - loss: 1.6641 - acc: 0.2934
Epoch 16/100
 - 0s - loss: 1.6742 - acc: 0.2688
Epoch 17/100
 - 0s - loss: 1.6750 - acc: 0.3035
Epoch 18/100
 - 0s - loss: 1.6661 - acc: 0.3124
Epoch 19/100
 - 0s - loss: 1.6621 - acc: 0.2889
Epoch 20/100
 - 0s - loss: 1.6036 - acc: 0.2945
Epoch 21/100
 - 0s - loss: 1.5638 - acc: 0.3124
E

 - 0s - loss: 1.4183 - acc: 0.3527
Epoch 73/100
 - 0s - loss: 1.3837 - acc: 0.3516
Epoch 74/100
 - 0s - loss: 1.3710 - acc: 0.3550
Epoch 75/100
 - 0s - loss: 1.3937 - acc: 0.3550
Epoch 76/100
 - 0s - loss: 1.3993 - acc: 0.3427
Epoch 77/100
 - 0s - loss: 1.4253 - acc: 0.3606
Epoch 78/100
 - 0s - loss: 1.3496 - acc: 0.3651
Epoch 79/100
 - 0s - loss: 1.4032 - acc: 0.3539
Epoch 80/100
 - 0s - loss: 1.3630 - acc: 0.3651
Epoch 81/100
 - 0s - loss: 1.3451 - acc: 0.3483
Epoch 82/100
 - 0s - loss: 1.3555 - acc: 0.3527
Epoch 83/100
 - 0s - loss: 1.3657 - acc: 0.3651
Epoch 84/100
 - 0s - loss: 1.3798 - acc: 0.3449
Epoch 85/100
 - 0s - loss: 1.3648 - acc: 0.3774
Epoch 86/100
 - 0s - loss: 1.4643 - acc: 0.3483
Epoch 87/100
 - 0s - loss: 1.3502 - acc: 0.3550
Epoch 88/100
 - 0s - loss: 1.3905 - acc: 0.3606
Epoch 89/100
 - 0s - loss: 1.3334 - acc: 0.3595
Epoch 90/100
 - 0s - loss: 1.3665 - acc: 0.3998
Epoch 91/100
 - 0s - loss: 1.3720 - acc: 0.3863
Epoch 92/100
 - 0s - loss: 1.3361 - acc: 0.3931
Epoch

 - 1s - loss: 0.3398 - acc: 0.8667
Epoch 44/100
 - 0s - loss: 0.3251 - acc: 0.8768
Epoch 45/100
 - 0s - loss: 0.3451 - acc: 0.8589
Epoch 46/100
 - 0s - loss: 0.3581 - acc: 0.8712
Epoch 47/100
 - 0s - loss: 0.3650 - acc: 0.8499
Epoch 48/100
 - 0s - loss: 0.3286 - acc: 0.8802
Epoch 49/100
 - 0s - loss: 0.3649 - acc: 0.8589
Epoch 50/100
 - 0s - loss: 0.3639 - acc: 0.8735
Epoch 51/100
 - 0s - loss: 0.2516 - acc: 0.9138
Epoch 52/100
 - 0s - loss: 0.2394 - acc: 0.9138
Epoch 53/100
 - 0s - loss: 0.2086 - acc: 0.9171
Epoch 54/100
 - 0s - loss: 0.2330 - acc: 0.9261
Epoch 55/100
 - 0s - loss: 0.2298 - acc: 0.9048
Epoch 56/100
 - 0s - loss: 0.2962 - acc: 0.8970
Epoch 57/100
 - 0s - loss: 0.3212 - acc: 0.8679
Epoch 58/100
 - 0s - loss: 0.3517 - acc: 0.8791
Epoch 59/100
 - 0s - loss: 0.2573 - acc: 0.9216
Epoch 60/100
 - 1s - loss: 0.1851 - acc: 0.9306
Epoch 61/100
 - 0s - loss: 0.1725 - acc: 0.9418
Epoch 62/100
 - 0s - loss: 0.1699 - acc: 0.9384
Epoch 63/100
 - 0s - loss: 0.2231 - acc: 0.9272
Epoch

 - 0s - loss: 0.9871 - acc: 0.5671
Epoch 15/100
 - 0s - loss: 0.9652 - acc: 0.5660
Epoch 16/100
 - 0s - loss: 0.9449 - acc: 0.5951
Epoch 17/100
 - 0s - loss: 0.9669 - acc: 0.5984
Epoch 18/100
 - 0s - loss: 0.9593 - acc: 0.5861
Epoch 19/100
 - 0s - loss: 0.9242 - acc: 0.6309
Epoch 20/100
 - 0s - loss: 0.8753 - acc: 0.6197
Epoch 21/100
 - 0s - loss: 0.8549 - acc: 0.6409
Epoch 22/100
 - 0s - loss: 0.8295 - acc: 0.6488
Epoch 23/100
 - 0s - loss: 0.7462 - acc: 0.6868
Epoch 24/100
 - 1s - loss: 0.7440 - acc: 0.6924
Epoch 25/100
 - 0s - loss: 0.7117 - acc: 0.6957
Epoch 26/100
 - 0s - loss: 0.7224 - acc: 0.7036
Epoch 27/100
 - 0s - loss: 0.8021 - acc: 0.6723
Epoch 28/100
 - 0s - loss: 0.6386 - acc: 0.7371
Epoch 29/100
 - 0s - loss: 0.6252 - acc: 0.7327
Epoch 30/100
 - 0s - loss: 0.5736 - acc: 0.7729
Epoch 31/100
 - 0s - loss: 0.5248 - acc: 0.7740
Epoch 32/100
 - 0s - loss: 0.5942 - acc: 0.7718
Epoch 33/100
 - 0s - loss: 0.6244 - acc: 0.7673
Epoch 34/100
 - 0s - loss: 0.4880 - acc: 0.8087
Epoch

Epoch 86/100
 - 0s - loss: 0.1113 - acc: 0.9574
Epoch 87/100
 - 0s - loss: 0.1227 - acc: 0.9597
Epoch 88/100
 - 0s - loss: 0.1163 - acc: 0.9597
Epoch 89/100
 - 0s - loss: 0.1224 - acc: 0.9597
Epoch 90/100
 - 0s - loss: 0.1149 - acc: 0.9619
Epoch 91/100
 - 0s - loss: 0.0960 - acc: 0.9630
Epoch 92/100
 - 0s - loss: 0.1152 - acc: 0.9630
Epoch 93/100
 - 0s - loss: 0.1236 - acc: 0.9541
Epoch 94/100
 - 0s - loss: 0.1044 - acc: 0.9642
Epoch 95/100
 - 0s - loss: 0.1130 - acc: 0.9574
Epoch 96/100
 - 0s - loss: 0.1196 - acc: 0.9563
Epoch 97/100
 - 0s - loss: 0.1081 - acc: 0.9597
Epoch 98/100
 - 0s - loss: 0.1077 - acc: 0.9630
Epoch 99/100
 - 0s - loss: 0.1067 - acc: 0.9597
Epoch 100/100
 - 0s - loss: 0.1009 - acc: 0.9664
Epoch 1/100
 - 21s - loss: 1.9355 - acc: 0.2665
Epoch 2/100
 - 0s - loss: 3.2566 - acc: 0.2688
Epoch 3/100
 - 0s - loss: 6.9090 - acc: 0.2475
Epoch 4/100
 - 1s - loss: 6.8402 - acc: 0.3449
Epoch 5/100
 - 0s - loss: 5.5573 - acc: 0.2217
Epoch 6/100
 - 0s - loss: 2.0713 - acc: 0.3

...

Epoch 70/100
 - 1s - loss: 0.2097 - acc: 0.9261
Epoch 71/100
 - 1s - loss: 0.1936 - acc: 0.9306
Epoch 72/100
 - 1s - loss: 0.1679 - acc: 0.9395
Epoch 73/100
 - 1s - loss: 0.2004 - acc: 0.9283
Epoch 74/100
 - 1s - loss: 0.1815 - acc: 0.9339
Epoch 75/100
 - 1s - loss: 0.1781 - acc: 0.9317
Epoch 76/100
 - 1s - loss: 0.1624 - acc: 0.9362
Epoch 77/100
 - 1s - loss: 0.1694 - acc: 0.9373
Epoch 78/100
 - 1s - loss: 0.1690 - acc: 0.9429
Epoch 79/100
 - 1s - loss: 0.1422 - acc: 0.9451
Epoch 80/100
 - 1s - loss: 0.1317 - acc: 0.9541
Epoch 81/100
 - 1s - loss: 0.1508 - acc: 0.9507
Epoch 82/100
 - 1s - loss: 0.1400 - acc: 0.9597
Epoch 83/100
 - 1s - loss: 0.1293 - acc: 0.9574
Epoch 84/100
 - 1s - loss: 0.1255 - acc: 0.9507
Epoch 85/100
 - 1s - loss: 0.1178 - acc: 0.9574
Epoch 86/100
 - 1s - loss: 0.1224 - acc: 0.9597
Epoch 87/100
 - 1s - loss: 0.1269 - acc: 0.9552
Epoch 88/100
 - 1s - loss: 0.1220 - acc: 0.9664
Epoch 89/100
 - 1s - loss: 0.1149 - acc: 0.9630
Epoch 90/100
 - 1s - loss: 0.1014 - acc:

Epoch 41/100
 - 1s - loss: 0.6997 - acc: 0.6999
Epoch 42/100
 - 1s - loss: 0.6327 - acc: 0.7268
Epoch 43/100
 - 1s - loss: 0.6425 - acc: 0.7324
Epoch 44/100
 - 1s - loss: 0.6334 - acc: 0.7256
Epoch 45/100
 - 0s - loss: 0.6370 - acc: 0.7458
Epoch 46/100
 - 1s - loss: 0.6400 - acc: 0.7335
Epoch 47/100
 - 1s - loss: 0.5719 - acc: 0.7772
Epoch 48/100
 - 1s - loss: 0.5724 - acc: 0.7772
Epoch 49/100
 - 1s - loss: 0.5613 - acc: 0.7704
Epoch 50/100
 - 0s - loss: 0.5819 - acc: 0.7570
Epoch 51/100
 - 0s - loss: 0.5591 - acc: 0.7626
Epoch 52/100
 - 0s - loss: 0.5574 - acc: 0.7727
Epoch 53/100
 - 0s - loss: 0.5360 - acc: 0.7884
Epoch 54/100
 - 0s - loss: 0.5228 - acc: 0.7951
Epoch 55/100
 - 0s - loss: 0.5138 - acc: 0.7996
Epoch 56/100
 - 0s - loss: 0.4700 - acc: 0.8119
Epoch 57/100
 - 0s - loss: 0.4474 - acc: 0.8331
Epoch 58/100
 - 0s - loss: 0.4217 - acc: 0.8343
Epoch 59/100
 - 0s - loss: 0.4561 - acc: 0.8175
Epoch 60/100
 - 0s - loss: 0.4403 - acc: 0.8175
Epoch 61/100
 - 1s - loss: 0.4375 - acc:

 - 0s - loss: 1.0768 - acc: 0.5089
Epoch 13/100
 - 0s - loss: 1.0646 - acc: 0.5246
Epoch 14/100
 - 0s - loss: 1.0472 - acc: 0.5403
Epoch 15/100
 - 0s - loss: 1.0445 - acc: 0.5101
Epoch 16/100
 - 0s - loss: 1.0274 - acc: 0.5414
Epoch 17/100
 - 0s - loss: 1.0144 - acc: 0.5470
Epoch 18/100
 - 0s - loss: 0.9638 - acc: 0.5626
Epoch 19/100
 - 0s - loss: 0.9555 - acc: 0.5738
Epoch 20/100
 - 0s - loss: 0.9894 - acc: 0.5694
Epoch 21/100
 - 0s - loss: 0.9215 - acc: 0.5872
Epoch 22/100
 - 0s - loss: 0.9239 - acc: 0.5895
Epoch 23/100
 - 0s - loss: 0.9148 - acc: 0.5906
Epoch 24/100
 - 0s - loss: 0.9034 - acc: 0.6096
Epoch 25/100
 - 0s - loss: 0.8684 - acc: 0.6040
Epoch 26/100
 - 0s - loss: 0.8712 - acc: 0.6040
Epoch 27/100
 - 0s - loss: 0.8133 - acc: 0.6387
Epoch 28/100
 - 0s - loss: 0.8249 - acc: 0.6342
Epoch 29/100
 - 0s - loss: 0.8010 - acc: 0.6555
Epoch 30/100
 - 0s - loss: 0.7897 - acc: 0.6465
Epoch 31/100
 - 0s - loss: 0.7951 - acc: 0.6566
Epoch 32/100
 - 0s - loss: 0.7593 - acc: 0.6678
Epoch

 - 0s - loss: 0.0967 - acc: 0.9653
Epoch 84/100
 - 0s - loss: 0.1157 - acc: 0.9597
Epoch 85/100
 - 0s - loss: 0.1085 - acc: 0.9630
Epoch 86/100
 - 0s - loss: 0.1214 - acc: 0.9563
Epoch 87/100
 - 0s - loss: 0.1127 - acc: 0.9608
Epoch 88/100
 - 0s - loss: 0.1123 - acc: 0.9541
Epoch 89/100
 - 0s - loss: 0.1119 - acc: 0.9630
Epoch 90/100
 - 0s - loss: 0.1082 - acc: 0.9530
Epoch 91/100
 - 0s - loss: 0.1240 - acc: 0.9563
Epoch 92/100
 - 0s - loss: 0.1121 - acc: 0.9586
Epoch 93/100
 - 0s - loss: 0.1114 - acc: 0.9574
Epoch 94/100
 - 0s - loss: 0.1051 - acc: 0.9574
Epoch 95/100
 - 0s - loss: 0.1159 - acc: 0.9597
Epoch 96/100
 - 0s - loss: 0.0897 - acc: 0.9642
Epoch 97/100
 - 0s - loss: 0.0868 - acc: 0.9642
Epoch 98/100
 - 0s - loss: 0.0814 - acc: 0.9653
Epoch 99/100
 - 0s - loss: 0.1011 - acc: 0.9597
Epoch 100/100
 - 0s - loss: 0.0727 - acc: 0.9698
Epoch 1/100
 - 3s - loss: 1.8710 - acc: 0.2419
Epoch 2/100
 - 0s - loss: 2.1120 - acc: 0.3337
Epoch 3/100
 - 0s - loss: 2.3683 - acc: 0.3169
Epoch 4

Epoch 55/100
 - 0s - loss: 0.1823 - acc: 0.9508
Epoch 56/100
 - 0s - loss: 0.1543 - acc: 0.9452
Epoch 57/100
 - 0s - loss: 0.1628 - acc: 0.9564
Epoch 58/100
 - 0s - loss: 0.1569 - acc: 0.9441
Epoch 59/100
 - 0s - loss: 0.1814 - acc: 0.9474
Epoch 60/100
 - 0s - loss: 0.1548 - acc: 0.9418
Epoch 61/100
 - 0s - loss: 0.1610 - acc: 0.9497
Epoch 62/100
 - 0s - loss: 0.1606 - acc: 0.9452
Epoch 63/100
 - 0s - loss: 0.1593 - acc: 0.9564
Epoch 64/100
 - 0s - loss: 0.1387 - acc: 0.9586
Epoch 65/100
 - 0s - loss: 0.1229 - acc: 0.9597
Epoch 66/100
 - 0s - loss: 0.1245 - acc: 0.9664
Epoch 67/100
 - 0s - loss: 0.1203 - acc: 0.9664
Epoch 68/100
 - 0s - loss: 0.1258 - acc: 0.9664
Epoch 69/100
 - 0s - loss: 0.1146 - acc: 0.9709
Epoch 70/100
 - 0s - loss: 0.1159 - acc: 0.9653
Epoch 71/100
 - 0s - loss: 0.1268 - acc: 0.9664
Epoch 72/100
 - 0s - loss: 0.1122 - acc: 0.9687
Epoch 73/100
 - 0s - loss: 0.1190 - acc: 0.9687
Epoch 74/100
 - 0s - loss: 0.1057 - acc: 0.9732
Epoch 75/100
 - 0s - loss: 0.1101 - acc:

In [9]:
t = grid_result.best_estimator_

In [10]:
print(t.score(X_test, y_test_for_keras))

0.5982142686843872


In [11]:
t.model.save('../model/Keras_best_params_pos_fs.h5')